## **1. 문자 단위 RNN**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np

### **1) 훈련 데이터 전처리하기**

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

문자 집합을 생성하고, 각 문자에 고유한 정수를 부여합니다.

In [3]:
char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩
dic_size = len(char_dic)

In [4]:
print(char_dic) # 여기서는 공백도 하나의 원소

{' ': 0, 'r': 1, ',': 2, 'i': 3, 's': 4, 'y': 5, 'd': 6, "'": 7, 'l': 8, 'b': 9, 'm': 10, 'h': 11, 'o': 12, 'c': 13, 'a': 14, 'u': 15, 't': 16, 'w': 17, '.': 18, 'p': 19, 'f': 20, 'g': 21, 'k': 22, 'e': 23, 'n': 24}


각 문자에 정수가 부여되었으면, 총 25개의 문자가 존재합니다. 즉, 문자 집합의 크기가 25이며 입력을 원-핫 벡터로 사용할 것이기에 매 시점 들어갈 입력값의 크기이기도 합니다. 

이제 하이퍼파라미터를 설정합니다. hidden_size를 입력의 크기와 동일하게 했는데 이는 사용자의 선택으로 다른 값을 해도 무방합니다. 그리고 sequence_length라는 변수는 우리가 앞서 만든 샘플을 10개 단위로 끊어서 샘플을 만들기 위해 선언했습니다. 이는 뒤에서 더 자세히 알아보겠습니다.

In [5]:
# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10 # 임의의 숫자 지정
learning_rate = 0.1

다음은 임의의 지정한 sequence_length 값인 10의 단위로 샘플들을 잘라서 데이터를 만드는 모습을 보여줍니다. 

In [6]:
# 데이터 구성
x_data = []
y_data = []

for i in range(len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)
    
    x_data.append([char_dic[c] for c in x_str]) # x_str to index
    y_data.append([char_dic[c] for c in y_str]) # y_str to index

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

총 170개의 샘플이 생성되었습니다. 그리고 각 샘플의 각 문자들은 고유한 정수로 인코딩된 상태입니다. 첫번째 샘플의 입력 데이터와 레이블 데이터를 확인해보겠습니다.

In [7]:
print(x_data[0]) # if you wan에 해당됨
print(y_data[0]) # f you want에 해당됨

[3, 20, 0, 5, 12, 15, 0, 17, 14, 24]
[20, 0, 5, 12, 15, 0, 17, 14, 24, 16]


한 칸씩 shift된 시퀀스가 정상적으로 출력되는 것을 볼 수 있습니다 이제 입력 시퀀스에 대해서 원-핫 인코딩을 수행하고, 입력 데이터와 레이블 데이터를 텐서로 변환합니다.

In [8]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

C:\Users\DELPHI~1\AppData\Local\Temp/ipykernel_1584/2895777156.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  X = torch.FloatTensor(x_one_hot)


이제 훈련 데이터와 레이블 데이터의 크기를 확인해봅시다.

In [9]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


원-핫 인코딩된 결과를 보기 위해 첫 번째 샘플만 출력해보겠습니다.

In [10]:
print(X[0])

tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

레이블 데이터의 첫 번째 샘플도 출력하겠습니다.

In [11]:
print(Y[0])

tensor([20,  0,  5, 12, 15,  0, 17, 14, 24, 16])


위 레이블 시퀀스는 f you want에 해당됩니다.

## **2. 모델 구현하기**

모델은 앞서 실습한 문자 단위 RNN 챕터와 거의 동일합니다. 다만 이번에는 은닉층을 두 개 쌓습니다.

In [12]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
    
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [13]:
net = Net(dic_size, hidden_size, 2) # 이번에는 층을 두 개 쌓습니다.

nn.RNN()안에 num_layers라는 인자는 은닉층을 몇 개 쌓을지 결정합니다. 이번에는 2를 입력하여 은닉층을 두 개 쌓았습니다.

이제 비용 함수와 옵티마이저를 선언합니다.

In [14]:
criteriion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

모델의 입력을 넣어서 출력의 크기를 확인하겠습니다.

In [15]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([170, 10, 25])


이는 (배치 차원, 시점, 출력의 크기)입니다. 나중에 정확도를 측정할 땐 view를 이용하여 배치 차원과 시점 차원을 하나로 만들어 사용합니다. 즉, 모두 펼쳐서 계산합니다.

In [16]:
print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환

torch.Size([1700, 25])


차원이 (1700, 25)가 된 것을 확인할 수 있습니다. 레이블 데이터의 크기도 다시 확인해보겠습니다.

In [17]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


레이블 데이터는 (170, 10)의 크기를 가지는데, 마찬가지로 나중에 정확도를 측정할 때는 이걸 펼쳐서 계산하게 됩니다.

이제 옵티마이저와 손실 함수를 정의하여 학습하겠습니다.

In [18]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criteriion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    # result의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져온다
            predict_str += ''.join([char_set[t] for t in result])
        else: # 두번째부터는 마지막 글자만 반복 추가하면 된다
            predict_str += char_set[result[-1]]
    
    print(predict_str)

yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
 oooo ooo ooooooo oooooo   o o  oooo oooo ooo  o ooo ooo  ooooo ooo ooo  ooo ooo  oooo     ooo  ooo   oo oooo oooooooo  o  ooo  oooo  oooooo ooo  ooo oooooo     o     ooooooo ooo 
                                                                                n                                                                                                  
 mfh ln ouuuluulllulluullun mlmullrurllulmuuluuulluuluul lfuuunululluulluo uuullmuoununlmgu,llmlllmumuuuumllmulunurlmumlumlfuuu,ullumluluufulmuluuuluuluulumlluunurlflllulumulruumu
rorrrdr r d d l r   d  srsd r r rlr dlrs r lrd   l r rl r dlrldrrsso   r rso  lor rlr dlrrlrslrrlrr ssrr  slr r r rlrrr rrlllr d rl rsldrdlr rlor do rl r ll sr r rrdrldr ldl rllrr
 ooo too ta a a a o t t aiooa o aoa aoaa oaoai   ioa ai  ato io aaa oao    a  t toaoa a iioa atoioa 

처음에는 이상한 문자만 나오지만 갈수록 문장다운 문장이 나오는 것을 확인할 수 있습니다.